<a href="https://colab.research.google.com/github/jamesharper23/Rec/blob/main/Copy_of_bookRecommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
import re
import string
import random
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt



In [ ]:
from google.colab import files
uploaded = files.upload()

MessageError: ignored

In [ ]:
import io

df = pd.read_csv(io.StringIO(uploaded["report_title_description_2021-01-19.csv"].decode("utf-8")))
df.head()
df.shape


In [ ]:
import nltk
nltk.download('stopwords')

In [ ]:
def _removeNonAscii(s):
    return "".join(i for i in s if  ord(i)<128)
# Function for converting into lower case
def make_lower_case(text):
    return text.lower()
# Function for removing stop words
def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text
# Function for removing punctuation
def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text
#Function for removing the html tags
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

# Applying all the functions in description and storing as a cleaned_desc

df['cleaned_desc'] = df['Book Description'].apply(_removeNonAscii)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_html)
df['cleaned_desc'] = df.cleaned_desc.apply(func = make_lower_case)
df['cleaned_desc'] = df.cleaned_desc.apply(func = remove_stop_words)
df['cleaned_desc'] = df.cleaned_desc.apply(func=remove_punctuation)
df['cleaned_desc'][20]

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity



In [ ]:
def recommendedBooks(bookTitle, recommendationByTitle, ngram):
    
    data = df.copy()
    data.reset_index(level = 0, inplace = True) 
    
    # Convert the index into series
    indices = pd.Series(data.index, index = data['Book Title'])
    
    #Converting the book title into vectors and used bigram
    tf = TfidfVectorizer(analyzer='word', ngram_range=(ngram, ngram), min_df = 1, stop_words='english')
    if recommendationByTitle:
      print("byTitle")
      tfidf_matrix = tf.fit_transform(data['Book Title'])
    else: 
      tfidf_matrix = tf.fit_transform(data['cleaned_desc'])

    # Calculating the similarity measures based on Cosine Similarity
    sg = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    # Get the index corresponding to original_title
    idx = indices[bookTitle]
    # Get the pairwsie similarity scores 
    sig = list(enumerate(sg[idx]))
    # Sort the books
    sig = sorted(sig, key=lambda x: x[1], reverse=True)
    # Scores of the 5 most similar books 
    sig = sig[1:6]
    # # Book indicies
    book_indices = [i[0] for i in sig]
   
    # Top 5 book recommendation
    rec = data[['Book Title', 'Book ID']].iloc[book_indices]
       
    # It reads the top 5 recommend book url and print the images
    for i in rec['Book Title']:
        print(i)

In [ ]:
Aluminum Alloy Database

Aluminum
Alloy
Database

Aluminum Alloy
Alloy Database


Aluminum Alloy Database

In [ ]:
searchTerm = "Aluminum Alloy Database"
recommendedBooks(bookTitle=searchTerm,recommendationByTitle=False,ngram=2)

In [ ]:
recommendedBooks(bookTitle=searchTerm,recommendationByTitle=False,ngram=1)

In [ ]:
recommendedBooks(searchTerm,recommendationByTitle=True, ngram=2)

In [ ]:
recommendedBooks(searchTerm, recommendationByTitle=False,ngram=3)

In [ ]:
recommendedBooks(searchTerm, recommendationByTitle=True,ngram=3)